In [4]:
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd

In [3]:
squad_dev = load_dataset('squad_v2', split='validation')
squad_dev[0]

Found cached dataset squad_v2 (C:/Users/shzso/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


{'id': '56ddde6b9a695914005b9628',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {'text': ['France', 'France', 'France', 'France'],
  'answer_start': [159, 159, 159, 159]}}

In [7]:
import warnings
warnings.filterwarnings("ignore")

squad_df = pd.DataFrame()
for row in tqdm(squad_dev):
    squad_df = squad_df.append({
        'question': row['question'],
        'context': row['context'],
        'id': row['id']
    }, ignore_index=True)
squad_df.head()

100%|██████████| 11873/11873 [00:08<00:00, 1354.08it/s]


,question,context,id
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9629
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962a
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962b
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962c


In [8]:
no_dupe = squad_df.drop_duplicates(
    subset='context',
    keep='first'
)
# also drop question column
no_dupe = no_dupe.drop(columns=['question'])
# and give each context a slightly unique ID
no_dupe['id'] = no_dupe['id'] + 'con'
no_dupe.head()

,context,id
0,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628con
9,"The Norman dynasty had a major political, cult...",56dddf4066d3e219004dad5fcon
17,"The English name ""Normans"" comes from the Fren...",56dde0379a695914005b9636con
21,"In the course of the 10th century, the initial...",56dde0ba66d3e219004dad75con
28,"Before Rollo's arrival, its populations did no...",56dde1d966d3e219004dad8dcon


In [9]:
squad_df = squad_df.merge(no_dupe, how='inner', on='context')
squad_df.head()

,question,context,id_x,id_y
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628,56ddde6b9a695914005b9628con
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9629,56ddde6b9a695914005b9628con
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962a,56ddde6b9a695914005b9628con
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962b,56ddde6b9a695914005b9628con
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962c,56ddde6b9a695914005b9628con


In [13]:
ir_queries = {
    row['id_x']: row['question'] for i, row in squad_df.iterrows()
}

ir_corpus = {
    row['id_y']: row['context'] for i, row in squad_df.iterrows()
}

ir_relevant_docs = {key: [] for key in squad_df['id_x'].unique()}
for i, row in squad_df.iterrows():
    # we append in the case of a question ID being connected to
    # multiple context IDs
    ir_relevant_docs[row['id_x']].append(row['id_y'])
# this must be in format {question_id: {set of context_ids}}
ir_relevant_docs = {key: set(values) for key, values in ir_relevant_docs.items()}

from sentence_transformers.evaluation import InformationRetrievalEvaluator

ir_eval = InformationRetrievalEvaluator(
    ir_queries, ir_corpus, ir_relevant_docs
)


In [14]:
from sentence_transformers import SentenceTransformer
modelPath = 'D:\Codes\ODQA\mpnet-mnr-squad2'

model = SentenceTransformer(modelPath)

In [15]:
ir_eval(model=model)

0.6746186834154515

In [16]:
qa=SentenceTransformer('D:\Codes\ODQA\multi-qa-mpnet-base-cos-v1')
ir_eval(qa)

0.7611637354436034